# El tratamiento de las variables categóricas

In [183]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [214]:
df = pd.read_csv('../datasets/ecom-expense/Ecom Expense.csv')

In [215]:
df.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485


In [216]:
dummy_gender = pd.get_dummies(df["Gender"], prefix="Gender")
dummy_city_tier = pd.get_dummies(df["City Tier"],prefix="City")

In [217]:
dummy_gender.head()

,Gender_Female,Gender_Male
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0


In [218]:
dummy_city_tier.head()

,City_Tier 1,City_Tier 2,City_Tier 3
0,1,0,0
1,0,1,0
2,0,1,0
3,1,0,0
4,0,1,0


In [219]:
column_names = df.columns.values
column_names

array(['Transaction ID', 'Age ', ' Items ', 'Monthly Income',
       'Transaction Time', 'Record', 'Gender', 'City Tier', 'Total Spend'],
      dtype=object)

In [220]:
df_new = df[column_names].join(dummy_gender).join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,1,0,0,1,0


In [221]:
df_new = df_new.drop(["Gender", "City Tier"], axis=1)

In [222]:
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,4198.385084,1,0,1,0,0
1,TXN002,24,8,17747,126.904567,3,4134.976648,1,0,0,1,0
2,TXN003,47,11,22845,873.469701,2,5166.614455,0,1,0,1,0
3,TXN004,50,11,18552,380.219428,7,7784.447676,1,0,1,0,0
4,TXN005,60,2,14439,403.374223,2,3254.160485,1,0,0,1,0


In [223]:
feature_cols = ["Monthly Income", "Transaction Time", 
                "Gender_Female", "Gender_Male", 
                "City_Tier 1", "City_Tier 2", "City_Tier 3"
                , "Record"]

In [224]:
X = df_new[feature_cols]
y = df_new["Total Spend"]

In [225]:
lm = LinearRegression()
lm.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [226]:
print(lm.intercept_)
print(lm.coef_)

-79.4171303013718
[ 1.47538980e-01  1.54946125e-01 -1.31025013e+02  1.31025013e+02
  7.67643260e+01  5.51389743e+01 -1.31903300e+02  7.72233446e+02]


In [227]:
list_coef = list(zip(feature_cols, lm.coef_))
list_coef

[('Monthly Income', 0.14753898049205738),
 ('Transaction Time', 0.15494612549589634),
 ('Gender_Female', -131.02501325554624),
 ('Gender_Male', 131.02501325554607),
 ('City_Tier 1', 76.76432601049513),
 ('City_Tier 2', 55.1389743092325),
 ('City_Tier 3', -131.9033003197277),
 ('Record', 772.2334457445645)]

In [228]:
lm.score(X,y)

0.9179923586131016

El modelo puede ser escrito como:
    Total_Spend = -79.4171303013718 
        + `Monthly Income` * `0.14753898049205738`
        + `Transaction Time` *  `0.15494612549589634`
        + `Gender_Female` * `-131.02501325554624`
        + `Gender_Male` * `131.02501325554607`
        + `City_Tier 1` * `76.76432601049513`
        + `City_Tier 2` * `55.1389743092325`
        + `City_Tier 3` * `-131.9033003197277`
        + `Record` * `772.2334457445645)`
* Si es hombre y vive en CT1: Total_Spend = 128.37220896466724 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545+'Record'* 772.2334457445648
* Si es hombre y vive en CT2: Total_Spend = 106.74685726340445 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 +'Record'* 772.2334457445648
* Si es hombre y vive en CT3: Total_Spend = -80.29541736555583 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545+'Record'* 772.2334457445648
* Si es mujer y vive en CT1: Total_Spend = -79.41713030137362 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 - 131.0250132555456+ 76.76432601049527 +'Record'* 772.2334457445648
* Si es mujer y vive en CT2: Total_Spend = -79.41713030137362 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 - 131.0250132555456+ 55.138974309232474 +'Record'* 772.2334457445648
* Si es mujer y vive en CT3: Total_Spend = -79.41713030137362 + 'Monthly Income' 0.14753898049205738 + 'Transaction Time' 0.15494612549589545 - 131.0250132555456-131.9033003197278 +'Record'* 772.2334457445648


In [229]:
prediction = -79.4171303013718
for i in list_coef:
    prediction += df_new[i[0]] * i[1]

In [230]:
df_new["prediction"] = prediction

In [231]:
df_new.head(20)

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction
0,TXN001,42,10,7313,627.668127,5,4198.385084,1,0,1,0,0,4903.696720
1,TXN002,24,8,17747,126.904567,3,4134.976648,1,0,0,1,0,4799.434826
2,TXN003,47,11,22845,873.469701,2,5166.614455,0,1,0,1,0,5157.082504
3,TXN004,50,11,18552,380.219428,7,7784.447676,1,0,1,0,0,8068.012996
4,TXN005,60,2,14439,403.374223,2,3254.160485,1,0,0,1,0,3581.980335
5,TXN006,49,6,6282,48.974268,2,2375.036467,0,1,0,1,0,2585.641997
6,TXN007,21,14,7086,961.203768,8,7494.474559,0,1,1,0,0,7500.635790
7,TXN008,58,9,8881,962.253740,10,10782.944920,0,1,0,0,1,9101.430215
8,TXN009,20,6,5635,858.328131,5,3854.277411,0,1,1,0,0,4953.916211
9,TXN010,48,12,20861,43.036737,4,5346.140262,1,0,0,1,0,6018.109661


In [232]:
SSD = np.sum((df_new["prediction"] - df_new["Total Spend"])**2)

In [233]:
SSD

1517733985.340816

In [234]:
RSE = np.sqrt(SSD/(len(df_new)-len(feature_cols)-1))

In [235]:
RSE

803.1318809818165

In [236]:
sales_mean = np.mean(df_new["Total Spend"])
sales_mean

6163.176415976715

In [237]:
error = RSE/sales_mean
error*100

13.03113568029416

In [238]:
df_new["prediction_with_sklearn"] = lm.predict(pd.DataFrame(df_new[feature_cols]))

In [240]:
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Total Spend,Gender_Female,Gender_Male,City_Tier 1,City_Tier 2,City_Tier 3,prediction,prediction_with_sklearn
0,TXN001,42,10,7313,627.668127,5,4198.385084,1,0,1,0,0,4903.696720,4903.696720
1,TXN002,24,8,17747,126.904567,3,4134.976648,1,0,0,1,0,4799.434826,4799.434826
2,TXN003,47,11,22845,873.469701,2,5166.614455,0,1,0,1,0,5157.082504,5157.082504
3,TXN004,50,11,18552,380.219428,7,7784.447676,1,0,1,0,0,8068.012996,8068.012996
4,TXN005,60,2,14439,403.374223,2,3254.160485,1,0,0,1,0,3581.980335,3581.980335


# Eliminar variabes dummy redundantes

In [243]:
dummy_gender = pd.get_dummies(df["Gender"], prefix="Gender").iloc[:,1:]

In [244]:
dummy_gender.head()

,Gender_Male
0,0
1,0
2,1
3,0
4,0


In [245]:
dummy_city_tier = pd.get_dummies(df["City Tier"], prefix="City").iloc[:,1:]
dummy_city_tier

,City_Tier 2,City_Tier 3
0,0,0
1,1,0
2,1,0
3,0,0
4,1,0
...,...,...
2357,1,0
2358,1,0
2359,0,1
2360,0,0


In [247]:
df_new = df[column_names].join(dummy_gender).join(dummy_city_tier)
df_new.head()

,Transaction ID,Age,Items,Monthly Income,Transaction Time,Record,Gender,City Tier,Total Spend,Gender_Male,City_Tier 2,City_Tier 3
0,TXN001,42,10,7313,627.668127,5,Female,Tier 1,4198.385084,0,0,0
1,TXN002,24,8,17747,126.904567,3,Female,Tier 2,4134.976648,0,1,0
2,TXN003,47,11,22845,873.469701,2,Male,Tier 2,5166.614455,1,1,0
3,TXN004,50,11,18552,380.219428,7,Female,Tier 1,7784.447676,0,0,0
4,TXN005,60,2,14439,403.374223,2,Female,Tier 2,3254.160485,0,1,0


In [248]:
feature_cols = ["Monthly Income", "Transaction Time", 
                "Gender_Male", 
                "City_Tier 2", "City_Tier 3"
                , "Record"]

In [249]:
X = df_new[feature_cols]
y = df_new["Total Spend"]
lm = LinearRegression()
lm.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [250]:
lm.intercept_

-133.67781754642238

In [251]:
list(zip(feature_cols, lm.coef_))

[('Monthly Income', 0.14753898049205744),
 ('Transaction Time', 0.1549461254959002),
 ('Gender_Male', 262.0500265110948),
 ('City_Tier 2', -21.62535170126276),
 ('City_Tier 3', -208.66762633022296),
 ('Record', 772.2334457445636)]

In [252]:
lm.score(X,y)

0.9179923586131016